In [28]:
from __future__ import print_function

import librosa
import librosa.display
import IPython.display
import numpy as np
import pandas as pd
import math
import csv

In [36]:
audio_file = './data/audio/rap/16. Lay Me Bare.mp3'

collection_file = "./data/0_collection.csv"
ouput_folder = './data/db_a/'
rate = 44100
mono = True
n_components = 8
count_shrink_freq = 50000

In [31]:
df = pd.read_csv(collection_file, sep='\t')
df = df.drop(['Unnamed: 0'], axis=1)

In [6]:
def represent_audio(audio_file, rate, mono, n_components, count_shrink_freq):
    # Load the track in mono
    y, sr = librosa.load(audio_file, mono=mono, sr=rate)

    # Let's decompose a spectrogram with NMF, and then resynthesize an individual component
    D = librosa.stft(y)

    # Separate the magnitude and phase
    S, phase = librosa.magphase(D)

    # Decompose by nmf
    components, activations = librosa.decompose.decompose(S, n_components=n_components, sort=True)

    y_result = []

    for i in range(0, len(activations)):
        # Reconstruct a spectrogram by the outer product of component k and its activation
        D_k = np.multiply.outer(components[:, i], activations[i])

        # invert the stft after putting the phase back in
        y_k = librosa.istft(D_k * phase)
        y_k = y_k[::math.ceil(len(y_k)/count_shrink_freq)]

        y_result.append(y_k)
        
    return y_result

In [37]:
def save2csv(data, name, ouput_folder):
    with open(ouput_folder+name, 'w') as writeFile:
        writer = csv.writer(writeFile)
        writer.writerows(data)
    writeFile.close() 

In [45]:
curren_index = 0

start_index = 4
finish_index = 6 #df.shape[0]

for index, row in df.iterrows():
    if(curren_index>start_index and curren_index<=finish_index):
        #print(j)
        tty = represent_audio(row['filepath'], rate, mono, n_components, count_shrink_freq)

        output_file_name = str(row['i'])+".csv"
        save2csv(tty, output_file_name, ouput_folder)
        
    curren_index +=1
    